In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('device : ', device)

device :  cuda:0


In [3]:
# 0~1, -1~1
transform = transforms.Compose([
    # 0 ~ 1
    transforms.ToTensor(),
    # (0 - 0.5) / 0.5 --> -1 /  (1 -0.5)/0.5 --> 1 # -1~1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # (mean), (std) for each channel --> (data - mean) / std
    # flip 좌우반전
    transforms.RandomHorizontalFlip(p=0.5),
])

# 0~1, -1~1
transform_test = transforms.Compose([
    # 0 ~ 1
    transforms.ToTensor(),
    # (0 - 0.5) / 0.5 --> -1 /  (1 -0.5)/0.5 --> 1 # -1~1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # (mean), (std) for each channel --> (data - mean) / std
])

In [4]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

100%|██████████| 170498071/170498071 [00:13<00:00, 13080959.56it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [7]:
class simple_cnn(nn.Module):
  def __init__(self):
    #32/32/3
    super(simple_cnn, self).__init__()

    self.conv_layer = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=4, kernel_size=(3,3) ),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(3,3) ),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2 ),

        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3)),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3)),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # 5*5*16
    )
    # fc layer Linear

    self.fc_layer = nn.Sequential(
        nn.Linear(16*5*5, 64),
        nn.ReLU(inplace=True),
        nn.Linear(64, 32),
        nn.ReLU(inplace=True),
        nn.Linear(32, 10),
    )

  def forward(self, x):
    o = self.conv_layer(x)
    # 5x5x16 # batch size- channel, h, w
    o = o.view(o.shape[0], -1) #numpy reshape
    o = self.fc_layer(o)

    return o


In [8]:
cnn_net = simple_cnn()